# Random Number Generation

[Chaoming Wang](mailto:chao.brain@qq.com)

Although ``brainpy.math.random`` is designed to be seamlessly compatible with ``numpy.random``, there are still some differences under the context of JIT compilation.

In this section, we are going to talk about how to program a JIT-compatible code with ``brainpy.math.random``.

In [26]:
import brainpy as bp
import brainpy.math as bm
import numpy as np

In [27]:
# bm.set_platform('cpu')

## Using ``bm.random`` outside functions to JIT

Using ``brainpy.math.random`` outside of functions to JIT is the same as using ``numpy.random``.

This usage corresponds to the cases that generating random data for further processing. For example,

In [28]:
bm.random.rand(10)

Array([0.3848505 , 0.9607172 , 0.2464571 , 0.4974357 , 0.33016646,
          0.8089963 , 0.43412983, 0.94763744, 0.60578203, 0.8153539 ],            dtype=float32)

In [29]:
np.random.rand(10)

array([0.59098454, 0.2081954 , 0.8631013 , 0.54420917, 0.43476752,
       0.18384759, 0.62805408, 0.23767145, 0.75848286, 0.6764358 ])

When you are using API functions in ``brainpy.math.random``, actually you are calling functions in a default ``RandomState``. Similarly, ``numpy.random`` also has a default ``RandomState``. Calling a random function in ``numpy.random`` module corresponds to calling the random function in this default NumPy ``RandomState``.

In [30]:
bm.random.DEFAULT

RandomState([3161893838,  596844228], dtype=uint32)

## Using ``bm.random`` inside a function to JIT

If you are using random sampling in a JIT function, **there are things you need to pay attention to**. Otherwise, the error is likely to raise.

As I have stated above, ``brainpy.math.random`` functions are using the default ``RandomState``. A ``RandomState`` is an instance of brainpy [Variable](./arrays_and_variables.ipynb), denoting that it has values to change after calling any its built-in random function. What's changing is the `key` of a ``RandomState``. For instance,

In [31]:
bm.random.rand(1)
print('Now, the DEFAULT is', bm.random.DEFAULT)

Now, the DEFAULT is RandomState([ 204639563, 3885133714], dtype=uint32)


In [32]:
bm.random.rand(1)
print('Now, the DEFAULT is', bm.random.DEFAULT)

Now, the DEFAULT is RandomState([3315775143, 3891325380], dtype=uint32)


Therefore, if you do not specify this DEFAULT ``RandomState`` you are using, repeatedly calling random functions in ``brainpy.math.random`` module will not get what you want, because its `key` cannot be updated. For instance,

In [33]:
@bm.jit
def get_data():
    return bm.random.random(2)

In [34]:
get_data()

Array([0.11177993, 0.9642259 ], dtype=float32)

In [35]:
get_data()

Array([0.11177993, 0.9642259 ], dtype=float32)

A correct way is explicitly declaring you are using this DEFAULT variable in the JIT transformation.

In [36]:
bm.random.seed()

In [37]:
from functools import partial

@partial(bm.jit, dyn_vars=(bm.random.DEFAULT, ))
def get_data_v2():
    return bm.random.random(2)

In [38]:
get_data_v2()

Array([0.6374749 , 0.59050643], dtype=float32)

In [39]:
get_data_v2()

Array([0.7263355 , 0.08753729], dtype=float32)

## Using ``RandomState`` for objects to JIT

Another way I recommend is using instances of ``RandomState`` for objects to JIT. For example, you can initialize a ``RandomState`` in the ``__init__()`` function, then using the initialized ``RandomState`` anywhere.

In [40]:
class MyOb(bp.BrainPyObject):
    def __init__(self):
        super().__init__()
        self.rng = bm.random.RandomState(123)

    def __call__(self):
        size = (50, 100)
        u = self.rng.random(size)
        v = self.rng.uniform(size=size)
        z = bm.sqrt(-2 * bm.log(u)) * bm.cos(2 * bm.pi * v)
        return z

In [41]:
ob = bm.jit(MyOb())

In [42]:
ob()

Array([[ 1.3595979 , -1.3462192 ,  0.7149456 , ...,  1.4283268 ,
           -1.1362855 , -0.18378317],
          [-0.26401126, -1.6798397 , -0.8422355 , ...,  1.0795223 ,
            0.41247413, -0.955116  ],
          [ 0.6234829 , -0.44811824, -0.03835859, ..., -2.5203867 ,
           -0.02713326,  1.6490041 ],
          ...,
          [-0.9861029 ,  0.36676335, -0.31499916, ...,  1.526808  ,
           -0.7946268 , -0.86713606],
          [-1.7008592 , -0.05957834, -0.5677447 , ..., -0.04765594,
            0.574145  , -0.11830498],
          [-0.22663854, -1.8517947 , -1.3546717 , ...,  1.2332705 ,
           -0.79247886, -1.9352005 ]], dtype=float32)

Note that any ``Variable`` instance which can be directly accessed by ``self.`` is able to be automatically found by brainpy's JIT transformation functions.
Therefore, in this case, we do not need to pass the ``rng`` into the `dyn_vars` in ``bm.jit()`` function.